### MNIST로 딥러닝 스타트
* 학습 내용
  * 01. Tensorflow를 활용하여 MNIST 모델을 구현해 본다.
  * 02. 배치 사이즈에 대해 이해해 본다. 
  * 03. 활성화 함수에 대해 이해해 본다.

In [1]:
import tensorflow as tf

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:455: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:456: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:457: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/tensorflow/pyth

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


### 01-02 신경망 모델 구성하기
* MNIST의 손글씨는 28 X 28로 구성되어 있다.
* 784개의 특징으로 구성되어 있음.(784pixel)
* 레이블은 0~9까지의 10개의 분류. (숫자0부터 9까지)

In [3]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

### 미니배치의 이해
* 이미지를 하나씩 학습시키는 것보다 여러 개를 한꺼번에 학습시키는 쪽이 효과가 좋다.
* 많은 메모리와 높은 컴퓨터 성능이 필요하므로 일반적으로 데이터를 적당한 크기로 잘라서 학습시킨다. 미니배치라고 한다.
* tf.float32, [None, 784] => None의 자리에는 한번에 학습시킬 이미지의 개수를 지정하는 값이 들어감., 즉 배치 크기를 지정하는 자리이다.

### 신경망의 구성
 * 784(입력, 특징 개수) -> 
             256(첫번째 은닉층 뉴런) -> 256(두번째 은닉층 뉴런)
                                               -> 10(결과값 0~9 분류 개수 ) 

In [5]:
# 입력층 - 첫번째 은닉층
W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X, W1))

# 첫번째 은닉층 - 두번째 은닉층
W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1, W2))

# 마지막 은닉층 - 출력층
W3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L2, W3)

print(W3)
print(model)

<tf.Variable 'Variable_5:0' shape=(256, 10) dtype=float32_ref>
Tensor("MatMul_5:0", shape=(?, 10), dtype=float32)


### 설명
* tf.random_normal([784, 256], stddev=0.01) => 표준편차가 0.01인 정규 분포를 가지는 임의의 값으로 뉴런을 초기화
* tf.matmul(X, W1) : 각 계층으로 들어오는 입력값에 가중치를 곱하고, tf.nn.relu함수를 이용하여 활성화 함수를 ReLU를 사용.
* 마지막 계층인 model 텐서에 W3를 곱함으로 요소 10개짜리 배열이 출력된다.
* 10개의 요소는 0부터 9까지의 숫자를 나타낸다.
* 가장 큰 값을 가진 인덱스(결과 항목)이 예측 결과에 가까운 숫자이다.
* 출력층은 보통 활성화 함수를 사용하지 않는다.

In [6]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

### 세션 열기

In [7]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

### 배치 사이즈 지정
* 미니배치의 크기 100개 설정.
* 학습 데이터의 총 개수 train.num_examples 를 배치 크기로 나눈다.
* 총 배치 크기가 몇개인지 확인

In [8]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
total_batch

550

In [10]:
# MNIST 데이터 전체를 학습하는 일을 15번 반복함.
# 학습 데이터 전체를 한 바퀴를 도는 일을 에포크(epoch)라 한다.
for epoch in range(15):
    total_cost = 0

    for i in range(total_batch):
        # 배치 사이즈만큼 데이터 가져오기 
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # 입력값 : batch_xs, 출력값 : batch_ys
        # 최적화를 수행 후, 손실을 구한다.
        _ , cost_val = sess.run([optimizer, cost],
                                feed_dict =  {X: batch_xs, Y: batch_ys})

        # 총 손실 계산
        total_cost = total_cost + cost_val

    print("Epoch : ", "%4d" %(epoch + 1), 
                '평균 Cost = ', "{:.3f}".format(total_cost/total_batch))
    
print("최적화 완료!")

Epoch :     1 평균 Cost =  0.400
Epoch :     2 평균 Cost =  0.143
Epoch :     3 평균 Cost =  0.096
Epoch :     4 평균 Cost =  0.070
Epoch :     5 평균 Cost =  0.054
Epoch :     6 평균 Cost =  0.039
Epoch :     7 평균 Cost =  0.032
Epoch :     8 평균 Cost =  0.025
Epoch :     9 평균 Cost =  0.022
Epoch :    10 평균 Cost =  0.017
Epoch :    11 평균 Cost =  0.014
Epoch :    12 평균 Cost =  0.018
Epoch :    13 평균 Cost =  0.012
Epoch :    14 평균 Cost =  0.012
Epoch :    15 평균 Cost =  0.012
최적화 완료!


### 정확도 확인

In [11]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))   # 매번 결과가 맞는가 확인.
is_correct

<tf.Tensor 'Equal:0' shape=(?,) dtype=bool>

In [12]:
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [13]:
print('정확도 :', sess.run(accuracy, 
                       feed_dict = {X:mnist.test.images,
                                    Y:mnist.test.labels}))

정확도 : 0.9787


### REF 
* Activation Function(활성화 함수) : https://towardsdatascience.com/activation-functions-neural-networks-1cbd9f8d91d6